In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import csv
import scipy.stats as stats
from datetime import datetime
from matplotlib import gridspec
from scipy.signal import argrelmax
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pywt

meal1 = pd.read_csv(r"./Processed_Data/mealData1.csv", error_bad_lines=False)
meal2 = pd.read_csv(r"./Processed_Data/mealData2.csv", error_bad_lines=False)
meal3 = pd.read_csv(r"./Processed_Data/mealData3.csv", error_bad_lines=False)
meal4 = pd.read_csv(r"./Processed_Data/mealData4.csv", error_bad_lines=False)
meal5 = pd.read_csv(r"./Processed_Data/mealData5.csv", error_bad_lines=False)

meal2.columns = meal1.columns
meal3.columns = meal1.columns
meal4.columns = meal1.columns
meal5.columns = meal1.columns

#meal dataframe - all five meal data files appended together
meal = pd.concat([meal1, meal2, meal3, meal4, meal5], ignore_index=True, sort = False)

no_meal1 = pd.read_csv(r"./Processed_Data/Nomeal1.csv", error_bad_lines=False)
no_meal2 = pd.read_csv(r"./Processed_Data/Nomeal2.csv", error_bad_lines=False)
no_meal3 = pd.read_csv(r"./Processed_Data/Nomeal3.csv", error_bad_lines=False)
no_meal4 = pd.read_csv(r"./Processed_Data/Nomeal4.csv", error_bad_lines=False)
no_meal5 = pd.read_csv(r"./Processed_Data/Nomeal5.csv", error_bad_lines=False)

no_meal2.columns = no_meal1.columns
no_meal3.columns = no_meal1.columns
no_meal4.columns = no_meal1.columns
no_meal5.columns = no_meal1.columns

#no meal dataframe - all five no meal data files appended together
no_meal = pd.concat([no_meal1, no_meal2, no_meal3, no_meal4, no_meal5], ignore_index=True, sort = False)


In [11]:
#Extract Monitoring CGM Trend
def extract_cgm_trend(df, result_df):

    lunch = [[]]
    means = []

    for i in range(1, len(df)):
        lunch.append(df.iloc[i])

    for i in range(0, len(lunch)):
        means.append(df.iloc[i].mean())

    countmaster = []

    for i in range(0, len(lunch)):
        count = 0

        for j in df.iloc[i]:
            if j < means[i]:    
                count += 1

        countmaster.append(count)

    percentage=[]
    for i in countmaster:
        percentage.append((i / len(df.iloc[0])) * 100)

    result_df['feature_2'] = np.asarray(percentage)
    
    #return result_df


In [12]:
#Extract Accelaration
def extract_acceleration(df, result_df):
    d=[]
    q=[]
    f=[]

    acc1 = []
    acc2 = []
    acc3 = []
    acc4 = []

    for j in range(0, df.shape[0]):
        b = df.iloc[j]
        d = []
        for i in range(len(b)):
            if(np.isnan(b[i])):
                continue
            else:
                d.append(b[i])

        if(len(d) >= 1):

            solar_elevation_angle_1stdev = np.gradient(d)

            solar_elevation_angle_2nddev = np.gradient(solar_elevation_angle_1stdev)

            q = solar_elevation_angle_2nddev
            arr = q[5:10]
            acc1.append(np.mean(arr))
            arr = q[10:15]
            acc2.append(np.mean(arr))
            arr = q[15:20]
            acc3.append(np.mean(arr))
            arr = q[20:25]
            acc4.append(np.mean(arr))

        else:
            acc1.append(0)
            acc2.append(0)
            acc3.append(0)
            acc4.append(0)

    result_df['acc1'] = acc1
    result_df['acc2'] = acc2
    result_df['acc3'] = acc3
    result_df['acc4'] = acc4
    
    #return result_df


In [13]:
#Extract Entropy
def SampEn(U, m, r):
    def _maxdist(x_i, x_j):
        result = max([abs(ua - va) for ua, va in zip(x_i, x_j)])
        return result

    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = 0
        for i in range(len(x)):
            for j in range(len(x)):
                if i == j:
                    continue
                C += (_maxdist(x[i], x[j]) <= r)
        return C
    
    N = len(U)
    if(_phi(m+1) / _phi(m) == 0) :
        return 1
    return -np.log(float(_phi(m+1)) / float(_phi(m)))

def extract_entropy(df, result_df):
    no_of_rows = df.shape[0]
    no_of_cols = df.shape[1]
    
    result = df.T
    cols = []
    for i in range(0, no_of_rows) :
        cols.append(str(i))
    result.columns = cols 
    m = 2
    
    entropy = []
    for i in cols :
        r = 0.2 * np.std(result[i])
        entropy.append(SampEn(result[i], 2, r))

    result_df['entropy'] = entropy
    #return result_df

In [14]:
#Extract PolyFit
def extract_polyfit(df, result_df):
   
    colid = []
    no_of_rows = df.shape[0]
    no_of_cols = df.shape[1]

    result = df.T
    no_of_coefficients = 6
    cols = []
    for i in range(0, no_of_rows) :
        cols.append(str(i))
    result.columns = cols
    
    for i in range(0, no_of_cols) :
        colid.append(i + 1)
    x = np.array(colid)
    fftmatrix = []
    
    for i in range(0, no_of_rows) :
        y = np.array(result[str(i)])
        polyres = np.polyfit(x, y, no_of_coefficients-1)  
        fftmatrix.append(polyres.tolist())
    fftmatrix = np.array(fftmatrix)   
  
    for i in range(no_of_coefficients) :
        result_df['Coefficient'+str(i)] = fftmatrix[:, i]
    #return result_df


In [15]:
#Extract DWT
def extract_dwt(df, result_df):
    cA, cB = pywt.dwt(df, 'haar')
    cA_threshold = pywt.threshold(cA, np.std(cA)/2, mode='soft')
    cB_threshold = pywt.threshold(cB, np.std(cB)/2, mode='soft')
    
    reconstructed_signal = pywt.idwt(cA_threshold, cB_threshold, 'haar')
    
    feature_dwt_top8 = cA[:,:-8]
    #sorted in Ascending
    #print(feature_dwt_top8.shape)
    for i in range(8) :
        result_df['DWT'+str(i)] = feature_dwt_top8[:, i]

In [16]:
#1. Feature extraction
meal_features = pd.DataFrame()
no_meal_features = pd.DataFrame()

def extract_features(data, result_df) : 
    extract_cgm_trend(data, result_df)
    extract_acceleration(data, result_df)
    extract_entropy(data, result_df)
    extract_polyfit(data, result_df)
    extract_dwt(data, result_df)

extract_features(meal, meal_features)
#print(meal_features)
extract_features(no_meal, no_meal_features)
#print(no_meal_features)

In [17]:
#2. PCA
pca = PCA(n_components=10)
#Fit meal features
principalComponents = pca.fit(meal_features)
#Transform meal features
PCA_mealdata = pca.transform(meal_features)
#Tranform no-meal features
PCA_nomealdata = pca.transform(no_meal_features)

#Create feature matrix
Training_data = np.concatenate((PCA_mealdata, PCA_nomealdata), axis=0)

Training_labels = []
no_of_mealrows = meal.shape[0]
no_of_nomealrows = no_meal.shape[0]
for i in range(no_of_mealrows) :
    Training_labels.append(1)
for i in range(no_of_nomealrows) :
    Training_labels.append(0)


In [18]:
#3. Training classifiers

from sklearn import tree
from sklearn.model_selection import RepeatedKFold
import pickle
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pickle


X = pd.DataFrame(Training_data)
y = pd.DataFrame(Training_labels)
Z = pd.concat([X, y], axis=1, sort=False)

#Randomize data
Total_Data = Z.reindex(np.random.permutation(Z.index))

#K fold cross validation
kf = RepeatedKFold(n_splits=5, n_repeats=10)

#Decision Tree
for train_index, test_index in kf.split(Total_Data):
    x_train = Total_Data.iloc[train_index, 0:10]
    y_train = Total_Data.iloc[train_index, 10]
    x_test = Total_Data.iloc[test_index, 0:10]
    y_test = Total_Data.iloc[test_index, 10]

    clf = tree.DecisionTreeClassifier()
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    
    optacc = accuracy_score(y_test, y_pred)

    print("Decision tree - \n", classification_report(y_test, y_pred))

    filename = "decision_tree" + str(optacc) + ".sav"
    pickle.dump(clf, open(filename, 'wb'))

#Random Forest Classifier
for train_index, test_index in kf.split(Total_Data):
    x_train = Total_Data.iloc[train_index, 0:10]
    y_train = Total_Data.iloc[train_index, 10]
    x_test = Total_Data.iloc[test_index, 0:10]
    y_test = Total_Data.iloc[test_index, 10]
    
    rcf = RandomForestClassifier(n_estimators=32)
    rcf.fit(x_train,y_train)

    y_pred = rcf.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Random Forest - \n" , classification_report(y_test, y_pred))
    
    filename = 'random_forest'+ str(accuracy) +'.sav'
    pickle.dump(rcf, open(filename, 'wb'))

#Gradient Boosting Classifier
for train_index, test_index in kf.split(Total_Data):
    x_train = Total_Data.iloc[train_index, 0:10]
    y_train = Total_Data.iloc[train_index, 10]
    x_test = Total_Data.iloc[test_index, 0:10]
    y_test = Total_Data.iloc[test_index, 10]
    
    gb_clf = GradientBoostingClassifier(n_estimators = 32, learning_rate = 0.25, max_depth = 4)
    gb_clf.fit(x_train, y_train)
    y_pred = gb_clf.predict(x_test)

    accuracy = accuracy_score(y_test, y_pred)
    print("Gradient Booster - \n", classification_report(y_test, y_pred))
    filename = 'gradient_booster' + str(accuracy) + '.sav'
    pickle.dump(gb_clf, open(filename, 'wb'))
    
#KNN Classifier
for train_index, test_index in kf.split(Total_Data):
    x_train = Total_Data.iloc[train_index, 0:10]
    y_train = Total_Data.iloc[train_index, 10]
    x_test = Total_Data.iloc[test_index, 0:10]
    y_test = Total_Data.iloc[test_index, 10]
    
    knn = KNeighborsClassifier(n_neighbors = 5, p = 1)
    knn.fit(x_train, y_train)
    y_pred = knn.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    print("KNN Classifier - \n", classification_report(y_test, y_pred))
    filename = 'knn' + str(accuracy) + '.sav'
    pickle.dump(knn, open(filename, 'wb'))

Decision tree - 
               precision    recall  f1-score   support

           0       0.51      0.41      0.46        51
           1       0.48      0.58      0.53        48

    accuracy                           0.49        99
   macro avg       0.50      0.50      0.49        99
weighted avg       0.50      0.49      0.49        99

Decision tree - 
               precision    recall  f1-score   support

           0       0.57      0.55      0.56        49
           1       0.58      0.60      0.59        50

    accuracy                           0.58        99
   macro avg       0.58      0.58      0.58        99
weighted avg       0.58      0.58      0.58        99

Decision tree - 
               precision    recall  f1-score   support

           0       0.51      0.50      0.51        50
           1       0.50      0.51      0.51        49

    accuracy                           0.51        99
   macro avg       0.51      0.51      0.51        99
weighted avg       0

Decision tree - 
               precision    recall  f1-score   support

           0       0.47      0.47      0.47        45
           1       0.56      0.56      0.56        54

    accuracy                           0.52        99
   macro avg       0.51      0.51      0.51        99
weighted avg       0.52      0.52      0.52        99

Decision tree - 
               precision    recall  f1-score   support

           0       0.59      0.50      0.54        52
           1       0.53      0.62      0.57        47

    accuracy                           0.56        99
   macro avg       0.56      0.56      0.56        99
weighted avg       0.56      0.56      0.55        99

Decision tree - 
               precision    recall  f1-score   support

           0       0.57      0.51      0.54        49
           1       0.56      0.62      0.59        50

    accuracy                           0.57        99
   macro avg       0.57      0.57      0.56        99
weighted avg       0

Random Forest - 
               precision    recall  f1-score   support

           0       0.62      0.64      0.63        55
           1       0.53      0.52      0.53        44

    accuracy                           0.59        99
   macro avg       0.58      0.58      0.58        99
weighted avg       0.58      0.59      0.59        99

Random Forest - 
               precision    recall  f1-score   support

           0       0.60      0.60      0.60        47
           1       0.63      0.63      0.63        52

    accuracy                           0.62        99
   macro avg       0.62      0.62      0.62        99
weighted avg       0.62      0.62      0.62        99

Random Forest - 
               precision    recall  f1-score   support

           0       0.62      0.58      0.60        52
           1       0.57      0.62      0.59        47

    accuracy                           0.60        99
   macro avg       0.60      0.60      0.60        99
weighted avg       0

Random Forest - 
               precision    recall  f1-score   support

           0       0.69      0.66      0.67        56
           1       0.58      0.60      0.59        43

    accuracy                           0.64        99
   macro avg       0.63      0.63      0.63        99
weighted avg       0.64      0.64      0.64        99

Random Forest - 
               precision    recall  f1-score   support

           0       0.60      0.59      0.60        49
           1       0.61      0.62      0.61        50

    accuracy                           0.61        99
   macro avg       0.61      0.61      0.61        99
weighted avg       0.61      0.61      0.61        99

Random Forest - 
               precision    recall  f1-score   support

           0       0.57      0.62      0.59        47
           1       0.62      0.58      0.60        52

    accuracy                           0.60        99
   macro avg       0.60      0.60      0.60        99
weighted avg       0

Gradient Booster - 
               precision    recall  f1-score   support

           0       0.59      0.60      0.60        53
           1       0.53      0.52      0.53        46

    accuracy                           0.57        99
   macro avg       0.56      0.56      0.56        99
weighted avg       0.57      0.57      0.57        99

Gradient Booster - 
               precision    recall  f1-score   support

           0       0.52      0.47      0.49        51
           1       0.49      0.54      0.51        48

    accuracy                           0.51        99
   macro avg       0.51      0.51      0.50        99
weighted avg       0.51      0.51      0.50        99

Gradient Booster - 
               precision    recall  f1-score   support

           0       0.61      0.62      0.62        45
           1       0.68      0.67      0.67        54

    accuracy                           0.65        99
   macro avg       0.64      0.64      0.64        99
weighted av

Gradient Booster - 
               precision    recall  f1-score   support

           0       0.56      0.57      0.56        51
           1       0.53      0.52      0.53        48

    accuracy                           0.55        99
   macro avg       0.54      0.54      0.54        99
weighted avg       0.55      0.55      0.55        99

Gradient Booster - 
               precision    recall  f1-score   support

           0       0.56      0.50      0.53        46
           1       0.60      0.66      0.63        53

    accuracy                           0.59        99
   macro avg       0.58      0.58      0.58        99
weighted avg       0.58      0.59      0.58        99

Gradient Booster - 
               precision    recall  f1-score   support

           0       0.62      0.46      0.53        57
           1       0.46      0.62      0.53        42

    accuracy                           0.53        99
   macro avg       0.54      0.54      0.53        99
weighted av

KNN Classifier - 
               precision    recall  f1-score   support

           0       0.61      0.55      0.58        55
           1       0.50      0.57      0.53        44

    accuracy                           0.56        99
   macro avg       0.56      0.56      0.55        99
weighted avg       0.56      0.56      0.56        99

KNN Classifier - 
               precision    recall  f1-score   support

           0       0.65      0.58      0.61        52
           1       0.58      0.65      0.61        46

    accuracy                           0.61        98
   macro avg       0.61      0.61      0.61        98
weighted avg       0.62      0.61      0.61        98

KNN Classifier - 
               precision    recall  f1-score   support

           0       0.43      0.49      0.46        43
           1       0.56      0.50      0.53        56

    accuracy                           0.49        99
   macro avg       0.49      0.49      0.49        99
weighted avg     

KNN Classifier - 
               precision    recall  f1-score   support

           0       0.49      0.53      0.51        45
           1       0.58      0.54      0.56        54

    accuracy                           0.54        99
   macro avg       0.53      0.54      0.53        99
weighted avg       0.54      0.54      0.54        99

KNN Classifier - 
               precision    recall  f1-score   support

           0       0.60      0.49      0.54        53
           1       0.52      0.63      0.57        46

    accuracy                           0.56        99
   macro avg       0.56      0.56      0.56        99
weighted avg       0.56      0.56      0.55        99

KNN Classifier - 
               precision    recall  f1-score   support

           0       0.59      0.67      0.63        48
           1       0.64      0.57      0.60        51

    accuracy                           0.62        99
   macro avg       0.62      0.62      0.62        99
weighted avg     